In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json 

### Загрузка необходимых данных 

In [2]:
with open("data/test_object_lu.json", "r") as file:
    map_objects = json.load(file)
    
partition = cv2.imread('data/partition.jpg')
v_corners = np.load("data/v_corners.npy")
h_corners = np.load("data/h_corners.npy")
panorama = cv2.imread("data/image_1024_aligned_rgb.png")
depth = np.load("data/depth_map.npy")

In [3]:
print(depth.shape)
print(depth)
# Нормализация данных с камеры глубины
scaled_depth = ((depth - np.min(depth)) / (np.max(depth) - np.min(depth)) * 255).astype(np.uint8)
cv2.imshow('Depth Image', scaled_depth)
cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 1024)
[[1.2148892 1.2148892 1.2148892 ... 1.2148892 1.2148892 1.2148892]
 [1.2149348 1.2149348 1.2149348 ... 1.2149348 1.2149348 1.2149348]
 [1.2150264 1.2150264 1.2150264 ... 1.2150264 1.2150264 1.2150264]
 ...
 [1.6001883 1.6001883 1.6001883 ... 1.6001883 1.6001883 1.6001883]
 [1.6000677 1.6000677 1.6000677 ... 1.6000677 1.6000677 1.6000677]
 [1.6000075 1.6000075 1.6000075 ... 1.6000075 1.6000075 1.6000075]]


In [4]:
# Загрузить изображения
alpha = 0.5

# Преобразовать глубину в трехканальное изображение
depth_colored = cv2.applyColorMap((scaled_depth * 255).astype(np.uint8), cv2.COLORMAP_JET)

# Наложить изображения
overlay = cv2.addWeighted(panorama, alpha, depth_colored, 1 - alpha, 0)

# Вывести результат
cv2.imshow('Overlay', overlay)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
cv2.imshow("source_image", panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(v_corners)

[[  30.914364  222.41164 ]
 [  30.914364  298.64883 ]
 [ 363.15613   140.81924 ]
 [ 363.15613   392.54047 ]
 [ 672.78357   149.66058 ]
 [ 672.78357   383.4991  ]
 [ 801.5895    137.09825 ]
 [ 801.5895    396.26495 ]
 [ 845.40314    76.09795 ]
 [ 845.40314   451.50623 ]
 [1008.1279    221.95033 ]
 [1008.1279    299.23853 ]]


In [6]:
for i in range(0, len(v_corners) - 2, 2):
    print(v_corners[i], v_corners[i + 1], v_corners[i + 2], v_corners[i + 3])

[ 30.914364 222.41164 ] [ 30.914364 298.64883 ] [363.15613 140.81924] [363.15613 392.54047]
[363.15613 140.81924] [363.15613 392.54047] [672.78357 149.66058] [672.78357 383.4991 ]
[672.78357 149.66058] [672.78357 383.4991 ] [801.5895  137.09825] [801.5895  396.26495]
[801.5895  137.09825] [801.5895  396.26495] [845.40314  76.09795] [845.40314 451.50623]
[845.40314  76.09795] [845.40314 451.50623] [1008.1279   221.95033] [1008.1279   299.23853]


In [7]:
print(h_corners)

[[221.67708 221.43863 221.38431 ... 222.38773 223.78665 223.0257 ]
 [299.86584 298.79175 299.72705 ... 298.76703 297.46896 299.3693 ]]


In [8]:
print(panorama.shape)

(512, 1024, 3)


### Влоб

In [9]:
num_rows = 250
num_columns = 1
channels = 3  # для цветных изображений (BGR)
new_image = np.zeros((num_rows, num_columns, channels), dtype=np.uint8)
for i in range(h_corners.shape[1]):
    lower, upper = h_corners[:,i].astype(int)
    sliced = panorama[:, i][lower:upper]
    resized = cv2.resize(sliced, (3, 250)).reshape((250, 1, 3))
    new_image = np.hstack([new_image, resized])
    
cv2.imshow("new_image", new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Сначала убираем вертикальное искажение, затем коррекция перспективы

In [10]:
pts_plane = [[[0, 0], [0, 250], [602, 0], [602, 250]],
             [[0, 0], [0, 250], [205, 0], [205, 250]],
             [[0, 0], [0, 250], [164, 0], [164, 250]],
             [[0, 0], [0, 250], [40, 0], [40, 250]],
             [[0, 0], [0, 250], [438, 0], [438, 250]]]

for i in range(0, len(v_corners) - 2, 2):
    area_idx = i // 2
    # Координаты для ключевых точек областей на исходной панораме
    left_lower, left_upper, right_lower, right_upper = v_corners[i], v_corners[i + 1], v_corners[i + 2], v_corners[i + 3]
    # Нахождение новой высоты для каждого столбца области 
    v_corners = v_corners.astype(int)
    # Ширина области на исходной панораме
    width = int(abs(left_upper[0] - right_upper[0]))
    upper_sequence = np.linspace(left_upper[1], right_upper[1], width).astype(int)
    lower_sequence = np.linspace(left_lower[1], right_lower[1], width).astype(int)
    sequence = upper_sequence - lower_sequence
    
    panorama_copy = panorama.copy()
    for new_size, panorama_pos, seq_pos in zip(sequence.astype(int), range(left_lower[0], right_lower[0]), range(width)):
        lower, upper = h_corners[:,panorama_pos].astype(int)
        sliced = panorama_copy[:, panorama_pos][lower:upper]
        resized = cv2.resize(sliced, (3, new_size))
        panorama_copy[:, panorama_pos][lower_sequence[seq_pos]:upper_sequence[seq_pos]] = resized
        
    # Отображение результата
    cv2.imshow('new_panorama', panorama_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # Определение ключевых точек на панораме и плоскости
    pts_panorama = np.float32([left_lower, left_upper, right_lower, right_upper])
    
    # Вычисление матрицы преобразования
    matrix = cv2.getPerspectiveTransform(pts_panorama, np.float32(pts_plane[area_idx]))
    
    # Применение преобразования
    result = cv2.warpPerspective(panorama_copy, matrix, [pts_plane[area_idx][2][0], 250])
    
    # Отображение результата
    cv2.imshow('Projected Image', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

TypeError: 'numpy.float32' object cannot be interpreted as an integer

In [ ]:
concatenated_panorama = np.concatenate((partition, partition), axis=1)
cv2.imshow("Concatenated", concatenated_panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def depth_to_coordinates(depth_map, fx, fy, cx, cy):
    """
    Преобразует карту глубины в 3D координаты.

    Параметры:
    - depth_map: Карта глубины.
    - fx, fy: Фокусные расстояния камеры.
    - cx, cy: Координаты центра изображения.

    Возвращает массив 3D координат в мировой системе координат.
    """
    rows, cols = depth_map.shape
    coordinates = np.zeros((rows, cols, 3), dtype=np.float32)

    for y in range(rows):
        for x in range(cols):
            depth = depth_map[y, x]
            coordinates[y, x, 0] = (x - cx) * depth / fx
            coordinates[y, x, 1] = (y - cy) * depth / fy
            coordinates[y, x, 2] = depth

    return coordinates

 # Загрузка карты глубины
depth_map = depth

# Параметры камеры (пример значения)
fx = 500.0  # фокусное расстояние по горизонтали
fy = 500.0  # фокусное расстояние по вертикали
cx = depth_map.shape[1] / 2.0  # координата центра изображения по горизонтали
cy = depth_map.shape[0] / 2.0  # координата центра изображения по вертикали

# Преобразование карты глубины в 3D координаты
coordinates = depth_to_coordinates(depth_map, fx, fy, cx, cy)

# Вывод 3D координат первой точки
print("3D Coordinates of the first point:", coordinates)



In [ ]:
import open3d as o3d
import numpy as np

# Создание случайного облака точек для примера
np.random.seed(123)
points = np.random.rand(100, 3)

# Создание объекта облака точек
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points)

# Визуализация облака точек
o3d.visualization.draw_geometries([point_cloud])
# Сохранение облака точек
o3d.io.write_point_cloud("point_cloud.ply", point_cloud)

# Загрузка облака точек
loaded_point_cloud = o3d.io.read_point_cloud("point_cloud.ply")
o3d.visualization.draw_geometries([loaded_point_cloud])


In [14]:
import open3d as o3d
import numpy as np
import cv2

rgb_image = panorama
depth_image = depth

# Создание облака точек из данных камеры RGB-D
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color=o3d.geometry.Image(rgb_image), 
    depth=o3d.geometry.Image(depth_image)
)
fx = 500.0  # фокусное расстояние по горизонтали
fy = 500.0  # фокусное расстояние по вертикали
cx = depth.shape[1] / 2.0  # координата центра изображения по горизонтали
cy = depth.shape[0] / 2.0  # координата центра изображения по вертикали

intrinsics = o3d.camera.PinholeCameraIntrinsic(
    width=rgb_image.shape[1], height=rgb_image.shape[0],
    fx=fx, fy=fy, cx=cx, cy=cy
)

point_cloud = o3d.geometry.create_point_cloud_from_rgbd_image(rgbd_image, intrinsics)

# Визуализация облака точек
o3d.visualization.draw_geometries([point_cloud])

AttributeError: module 'open3d.cpu.pybind.geometry' has no attribute 'create_point_cloud_from_rgbd_image'